
# Chatbot Interactif avec ChatGPT
Ce notebook offre une interface utilisateur améliorée pour interagir avec ChatGPT sur la thématique de la communication réseau point à point.

### Fonctionnalités
1. Interface interactive et visuelle avec des widgets.
2. Affichage formaté des conversations avec ChatGPT.
3. Historique des conversations visualisable et téléchargeable.
4. Indicateur visuel pendant la génération des réponses.

---

## Étapes :
1. Configurez le fichier `chatBot.env` avec votre clé API OpenAI.
2. Lancez le notebook et commencez à poser vos questions.
3. Visualisez ou téléchargez l'historique des conversations si nécessaire.


In [1]:

from IPython.display import display, HTML

# Ajouter un bouton pour masquer/afficher le code
display(HTML('''
    <style>
        .input { display: none; }
        .toggle-button { margin: 10px; cursor: pointer; }
    </style>
    <div class="toggle-button" onclick="
        var code_cells = document.querySelectorAll('.input');
        code_cells.forEach(cell => {
            cell.style.display = cell.style.display === 'none' ? '' : 'none';
        });
    ">Afficher/Masquer le Code</div>
'''))


In [2]:

# Installation des bibliothèques nécessaires
import sys
import subprocess

def install(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

# Liste des dépendances
dependencies = ["ipywidgets", "openai", "python-dotenv"]

for package in dependencies:
    install(package)

print("Toutes les dépendances sont installées.")


Installation de python-dotenv...
Toutes les dépendances sont installées.


In [3]:

# Charger les clés API à partir d'un fichier chatBot.env
from dotenv import load_dotenv
import os

# Charger les variables d'environnement depuis chatBot.env
load_dotenv(dotenv_path='chatBot.env')

openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("La clé API OpenAI n'est pas définie dans le fichier chatBot.env.")


In [4]:

def verify_keys():
    """Vérifier la validité de la clé API pour OpenAI."""
    try:
        import openai
        openai.api_key = openai_api_key
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Vérification de clé API"},
                {"role": "user", "content": "Test de clé API OpenAI"}
            ]
        )
        print("Clé OpenAI valide.")
    except Exception as e:
        raise ValueError(f"Clé OpenAI invalide : {e}")

# Vérification de la clé
verify_keys()


Clé OpenAI valide.


In [5]:

import ipywidgets as widgets
from IPython.display import display, HTML
import openai
from datetime import datetime

# Variables globales
history = []

# Widgets
username_input = widgets.Text(
    value="Utilisateur",
    description="Nom :",
    layout=widgets.Layout(width="50%")
)

user_input = widgets.Textarea(
    placeholder="Posez une question à ChatGPT...",
    layout=widgets.Layout(width="100%", height="80px")
)

send_button = widgets.Button(
    description="Envoyer",
    button_style="success",
    icon="paper-plane"
)

clear_button = widgets.Button(
    description="Réinitialiser",
    button_style="warning",
    icon="refresh"
)

save_button = widgets.Button(
    description="Télécharger l'historique",
    button_style="info",
    icon="download"
)

output_area = widgets.Output(layout=widgets.Layout(border="1px solid black", padding="10px", width="100%"))
spinner = widgets.Label(value="")  # Indicateur pendant le chargement


# Fonction pour interagir avec ChatGPT
def interact_with_chatgpt(prompt, username):
    openai.api_key = openai_api_key
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Vous êtes un expert en communication réseau."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message['content']
    except Exception as e:
        return f"Erreur lors de la communication avec ChatGPT : {e}"


# Gestion des événements
def handle_send(_):
    prompt = user_input.value.strip()
    username = username_input.value.strip() or "Utilisateur"
    if not prompt:
        with output_area:
            print("Veuillez entrer une question.")
        return

    # Afficher le spinner
    spinner.value = "Génération en cours..."
    output_area.clear_output()

    # Interaction avec ChatGPT
    response = interact_with_chatgpt(prompt, username)

    # Mise à jour de l'affichage et de l'historique
    with output_area:
        print(f"**{username} :** {prompt}\n")
        print(f"**ChatGPT :** {response}\n")
    history.append({"user": username, "prompt": prompt, "response": response})

    # Réinitialiser le spinner et l'entrée utilisateur
    spinner.value = ""
    user_input.value = ""


def handle_clear(_):
    history.clear()
    output_area.clear_output()
    print("Historique réinitialisé.")


def handle_save(_):
    if not history:
        with output_area:
            print("Aucun historique à sauvegarder.")
        return

    filename = f"conversation_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(filename, 'w') as f:
        f.write("# Historique de la Conversation\n")
        for entry in history:
            f.write(f"**{entry['user']} :** {entry['prompt']}\n\n")
            f.write(f"**ChatGPT :** {entry['response']}\n\n")
    with output_area:
        print(f"Historique sauvegardé sous : {filename}")


# Associer les boutons aux fonctions
send_button.on_click(handle_send)
clear_button.on_click(handle_clear)
save_button.on_click(handle_save)

# Affichage des widgets
display(widgets.HBox([username_input, spinner]))
display(user_input, widgets.HBox([send_button, clear_button, save_button]), output_area)


Textarea(value='', layout=Layout(height='80px', width='100%'), placeholder='Posez une question à ChatGPT...')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…